### Source
* https://python.langchain.com/docs/tutorials/agents/
* https://python.langchain.com/docs/integrations/chat/azure_chat_openai/

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
load_dotenv()

import getpass

if "AZURE_OPENAI_API_KEY" not in os.environ:
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass(
        "Enter your AzureOpenAI API key: "
    )

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
# Setup tracing

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient  # type: ignore
from azure.monitor.opentelemetry import configure_azure_monitor

project_endpoint = os.getenv("AI_FOUNDRY_PROJ_ENDPOINT")
project_client = AIProjectClient(endpoint=project_endpoint, credential=DefaultAzureCredential()) # type: ignore
configure_azure_monitor(
    connection_string=project_client.telemetry.get_connection_string()
)

In [ ]:
import requests
from langchain_core.tools import tool

@tool
def search_wikipedia(term: str) -> str:
    """Search wikipedia for the term provided"""
    # https://en.wikipedia.org/wiki/Neanderthal
    url = f"https://en.wikipedia.org/wiki/{term}"
    search_response = requests.get(url)

    prompt = f"The user asked a question about {term}\nSummerize the following context:\n##Context\n {search_response.content}"
    llm_response = llm.invoke(prompt)
    return llm_response.content # type: ignore

In [ ]:
# Import relevant functionality
# from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = llm
search = TavilySearchResults(max_results=2)
tools = [search, search_wikipedia]
system_prompt = "You are a friendly and helpful customer service agent. Use tools to answer the question if appropriate"
agent_executor = create_react_agent(
    model,
    tools,
    prompt=system_prompt,
    checkpointer=memory,
)

In [ ]:
from dataclasses import dataclass

@dataclass
class AgentEvalData:
    query: list[dict[str, str | dict]] # type: ignore
    response: list[dict[str, str | dict]] # type: ignore
    tool_definitions: list[dict[str, str | dict]] # type: ignore

In [ ]:
# Use the LangChain agent
thread_id = "abc123"
config = { "configurable": { "thread_id": thread_id } }

def _get_datetime():
    from datetime import datetime, timezone
    utc_now = datetime.now(timezone.utc)
    return utc_now.isoformat()

def _get_tool_definitions(tool_func):
    definitions = {
        "name": tool_func.name,
        "description": tool_func.description,
        "parameters": {
            "type": "object",
            "properties": {},
        }
    }

    # tool_func.args is a dict, use for loop to get each key in it
    for key in tool_func.args:
        definitions["parameters"]["properties"][key] = {
            "type": tool_func.args[key]['type'],
            "description": tool_func.args[key]['description'] if "description" in tool_func.args[key] else "",
        }

    return definitions

def agent_run(message: str) -> AgentEvalData:
    agent_eval_data: AgentEvalData = AgentEvalData(
        query=[{ # type: ignore
            "role": "system",
            "content": system_prompt,
        }, {
            "createdAt": _get_datetime(),
            "role": "user",
            "content": [ # type: ignore
                {
                    "type": "text",
                    "text": message,
                }
            ]
        }],
        response=[], # type: ignore
        tool_definitions=[ # type: ignore
            _get_tool_definitions(search),
            _get_tool_definitions(search_wikipedia),
        ]
    )

    for step in agent_executor.stream(
        {"messages": [HumanMessage(content=message)]},
        config, # type: ignore
        stream_mode="values",
    ):
        # step["messages"][-1].pretty_print()
        for message in step["messages"]:
            message.pretty_print() # type: ignore

    return agent_eval_data

In [ ]:
agent_run("what is the weather in Redmond, WA?")

In [ ]:
agent_run("how about tomorrow?")

In [ ]:
# agent_eval_data = agent_run("Neanderthal")
agent_eval_data = agent_run("Hanging_Gardens_of_Babylon")
from dataclasses import asdict
import json
print(json.dumps(asdict(agent_eval_data), indent=2)) # type: ignore